In [ ]:
! pip install s3fs

In [1]:
import numpy as np
import pyspark
import s3fs
from datetime import datetime

In [2]:
sc = pyspark.SparkContext.getOrCreate()
ss = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
test_rdd = sc.textFile("s3a://msds630-kaggle-competition/sessions_events_modified_v2.csv/part-00000")\
               .map(lambda line : line.encode('ascii', 'ignore'))
test_rdd.count()

5999419

In [4]:
columns = ["session_id", "start_timestamp","time_zone","timezone_offset","previous_sessions",
           "user_created_stamp","is_first_session","country", "region","city","latitude",
           "longitude","local","os_name","session_index","device_id","user_id_hash",
           "num_events","num_purchases","weekend_purchase","is_purchase",'purchase_times',
           'sum_purchase','1week','2week']
for i in range(len(columns)):
    if columns[i] == "user_id_hash": 
        print(i)

16


In [5]:
def convertTime(ts_str):
    if ts_str != "":
        return datetime.utcfromtimestamp(float(ts_str)/1000.0).strftime('%Y-%m-%d %H:%M:%S')
    else:
        return ""
convertTime('1541638424150')

'2018-11-08 00:53:44'

In [6]:
new_columns = ["user_id_hash", "session_id", "start_timestamp","time_zone","timezone_offset","previous_sessions",
               "user_created_stamp","is_first_session","country", "region","city","latitude",
               "longitude","local","os_name","session_index","device_id",
               "num_events","num_purchases","weekend_purchase","is_purchase",'purchase_times',
               'sum_purchase','1week','2week']

In [7]:
test_rdd_1 = test_rdd.map(lambda x: x.split(","))
test_rdd_2 = test_rdd_1.map(lambda x: [x[16],x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],
                                       x[8],x[9],x[10],x[11],x[12],x[13],x[14],x[15],
                                       x[17],x[18],x[19],x[20],x[21],x[22],x[23],x[24]])
test_rdd_3 = test_rdd_2.map(lambda x: [x[0],x[1],convertTime(x[2]),x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],
                                       x[11],x[12],x[13],x[14],x[15],x[16],x[17],x[18],x[19],x[20],
                                       x[21],x[22],x[23],x[24]])
test_rdd_4 = test_rdd_3.filter(lambda x: x[2] != "")

In [8]:
train_rdd = test_rdd_4.filter(lambda x: x[2] != "").filter(lambda x: x[2] < "2018-12-02")
train_label = test_rdd_4.filter(lambda x: x[2] != "").filter(lambda x: x[2] > "2018-12-01")

## Compute Label

In [9]:
def findPur(purchase_list):
    if '1.0' in purchase_list:
        return 1.0
    else:
        return -1.0
    
findPur(["1.0", ""])

1.0

In [10]:
train_label_1 = train_label.map(lambda x: [x[0], x[23], x[24]])
train_label_1_wk = train_label_1.map(lambda x: (x[0], x[1]))
train_label_2_wk = train_label_1.map(lambda x: (x[0], x[2]))
train_label_1_wk_1 = train_label_1_wk.groupByKey()\
                                     .map(lambda x: [x[0], list(x[1])])\
                                     .map(lambda x: (x[0], findPur(x[1])))
train_label_2_wk_1 = train_label_2_wk.groupByKey()\
                                     .map(lambda x: [x[0], list(x[1])])\
                                     .map(lambda x: (x[0], findPur(x[1])))
train_label_2 = train_label_1_wk_1.join(train_label_2_wk_1)\
                                  .map(lambda x: [x[0], x[1][0], x[1][1]])

In [11]:
print(train_label_2.count())
print(train_label_2.take(1))
print(train_label_2.filter(lambda x: x[2]==1).take(1))
print(train_label_2.filter(lambda x: x[1]!=1 and x[2] == 1).take(1))

62825
[['d6cf11a892144d6d2b5d953a5dba12033b2041655aaf26647f1f253e079f7951', -1.0, -1.0]]
[['b3755eea4d7e2c03d0d6a2492b7baba5844fdc23da0e6419eca99d4b95aba7ab', 1.0, 1.0]]
[['4a459979ce845aca82f5ef93f3bf7344447b66c76f9318ebdb8d2fa8a516075b', -1.0, 1.0]]


## Compute Train Dataset / Test Dataset

In [12]:
target_drop_column_name_list = ['time_zone', 'timezone_offset', "is_first_session", 
                                "region", "city", "latitude",
                                "longitude", "local", "session_index"]

In [13]:
new_columns_4 = list(set(new_columns)-set(target_drop_column_name_list))
new_columns_4_1 = []
for item in new_columns:
    if item in new_columns_4:
        new_columns_4_1.append(item)
new_columns_4_1

['user_id_hash',
 'session_id',
 'start_timestamp',
 'previous_sessions',
 'user_created_stamp',
 'country',
 'os_name',
 'device_id',
 'num_events',
 'num_purchases',
 'weekend_purchase',
 'is_purchase',
 'purchase_times',
 'sum_purchase',
 '1week',
 '2week']

In [14]:
def floatSafe(value): # In case there are non-integer type to be converted.
    try:
        return float(value)
    except ValueError:
        return 0.0

In [15]:
test_rdd_4_1 = test_rdd_4.map(lambda x: [x[0],x[1],x[2],x[5],x[6],x[8],
                                         x[14],x[16],x[17],x[18],x[19],
                                         x[20],x[21],x[22],x[23],x[24]])
train_rdd_4_1 = train_rdd.map(lambda x: [x[0],x[1],x[2],x[5],x[6],x[8],
                                         x[14],x[16],x[17],x[18],x[19],
                                         x[20],x[21],x[22],x[23],x[24]])
test_rdd_4_2 = test_rdd_4_1.map(lambda x: [x[0], x[1], x[2], floatSafe(x[3]), convertTime(x[4]),
                                           x[5], x[6], x[7], floatSafe(x[8]), floatSafe(x[9]),
                                           x[10], x[11], x[12], floatSafe(x[13]), 
                                           x[14], x[15]])
train_rdd_4_2 = train_rdd_4_1.map(lambda x: [x[0], x[1], x[2], floatSafe(x[3]), convertTime(x[4]),
                                             x[5], x[6], x[7], floatSafe(x[8]), floatSafe(x[9]),
                                             x[10], x[11], x[12], floatSafe(x[13]), 
                                             x[14], x[15]])

In [16]:
def getMode(nat_list):
    if nat_list == []:
        return ""
    count_list = []
    elem_list = list(set(nat_list))
    for elem in elem_list:
        count_list.append((nat_list.count(elem), elem))
    return sorted(count_list, reverse = True)[0][1]

def getMax(num_event_list):
    if num_event_list == []:
        return 0.0
    return max(num_event_list)

In [17]:
# user_id_hash & session_id
# Compute num of session_id
train_rdd_4_2_ses = train_rdd_4_2.map(lambda x: (x[0], x[1]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], len(list(x[1]))))
test_rdd_4_2_ses = test_rdd_4_2.map(lambda x: (x[0], x[1]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], len(list(x[1]))))

# user_id_hash & previous_sessions
# compute mean_previous_sessions
train_rdd_4_2_pse = train_rdd_4_2.map(lambda x: (x[0], x[3]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], np.mean(list(x[1]))))
test_rdd_4_2_pse = test_rdd_4_2.map(lambda x: (x[0], x[3]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], np.mean(list(x[1]))))

# user_id_hash & user_created_stamp
# compute the unique value for user_created_stamp
train_rdd_4_2_ucs = train_rdd_4_2.map(lambda x: (x[0], x[4]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], list(set(x[1]))[0]))
test_rdd_4_2_ucs = test_rdd_4_2.map(lambda x: (x[0], x[4]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], list(set(x[1]))[0]))

# user_id_hash & country
# compute the country mode
train_rdd_4_2_nat = train_rdd_4_2.map(lambda x: (x[0], x[5]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], getMode(list(x[1]))))
test_rdd_4_2_nat = test_rdd_4_2.map(lambda x: (x[0], x[5]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], getMode(list(x[1]))))

# user_id_hash & os_name
# compute the os_name mode
train_rdd_4_2_osm = train_rdd_4_2.map(lambda x: (x[0], x[6]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], getMode(list(x[1]))))
test_rdd_4_2_osm = test_rdd_4_2.map(lambda x: (x[0], x[6]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], getMode(list(x[1]))))

# user_id_hash & device_id
# compute the unique device_id count
train_rdd_4_2_dic = train_rdd_4_2.map(lambda x: (x[0], x[7]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], len(set(x[1]))))
test_rdd_4_2_dic = test_rdd_4_2.map(lambda x: (x[0], x[7]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], len(set(x[1]))))

# user_id_hash & num_events
# compute the max num of events
train_rdd_4_2_mne = train_rdd_4_2.map(lambda x: (x[0], x[8]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], getMax(list(x[1]))))
test_rdd_4_2_mne = test_rdd_4_2.map(lambda x: (x[0], x[8]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], getMax(list(x[1]))))

# user_id_hash & num_purchase_sum
# compute the sum of num_purchase_sum
train_rdd_4_2_nps = train_rdd_4_2.map(lambda x: (x[0], x[9]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], sum(list(x[1]))))
test_rdd_4_2_nps = test_rdd_4_2.map(lambda x: (x[0], x[9]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], sum(list(x[1]))))

# user_id_hash & sum_purchase
# compute sum of puchase in dollar
train_rdd_4_2_spd = train_rdd_4_2.map(lambda x: (x[0], x[13]))\
                                 .groupByKey()\
                                 .map(lambda x: (x[0], sum(list(x[1]))))
test_rdd_4_2_spd = test_rdd_4_2.map(lambda x: (x[0], x[13]))\
                               .groupByKey()\
                               .map(lambda x: (x[0], sum(list(x[1]))))

In [18]:
# Merge 1
train_rdd_merge_1 = train_rdd_4_2_ses.join(train_rdd_4_2_pse)
test_rdd_merge_1 = test_rdd_4_2_ses.join(test_rdd_4_2_pse)

# Merge 2
train_rdd_merge_2 = train_rdd_merge_1.join(train_rdd_4_2_ucs)\
                                     .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))
test_rdd_merge_2 = test_rdd_merge_1.join(test_rdd_4_2_ucs)\
                                   .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))

# Merge 3
train_rdd_merge_3 = train_rdd_merge_2.join(train_rdd_4_2_nat)\
                                     .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))
test_rdd_merge_3 = test_rdd_merge_2.join(test_rdd_4_2_nat)\
                                   .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))

# Merge 4
train_rdd_merge_4 = train_rdd_merge_3.join(train_rdd_4_2_osm)\
                                     .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))
test_rdd_merge_4 = test_rdd_merge_3.join(test_rdd_4_2_osm)\
                                   .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))

# Merge 5
train_rdd_merge_5 = train_rdd_merge_4.join(train_rdd_4_2_dic)\
                                     .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))
test_rdd_merge_5 = test_rdd_merge_4.join(test_rdd_4_2_dic)\
                                   .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))

# Merge 6
train_rdd_merge_6 = train_rdd_merge_5.join(train_rdd_4_2_mne)\
                                     .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))
test_rdd_merge_6 = test_rdd_merge_5.join(test_rdd_4_2_mne)\
                                   .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))

# Merge 7
train_rdd_merge_7 = train_rdd_merge_6.join(train_rdd_4_2_nps)\
                                     .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))
test_rdd_merge_7 = test_rdd_merge_6.join(test_rdd_4_2_nps)\
                                   .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))

# Merge 8
train_rdd_merge_8 = train_rdd_merge_7.join(train_rdd_4_2_spd)\
                                     .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))
test_rdd_merge_8 = test_rdd_merge_7.join(test_rdd_4_2_spd)\
                                   .map(lambda x: (x[0], sum([x[1][0], (x[1][1],)], ())))

In [19]:
train_rdd_5 = train_rdd_merge_8.map(lambda x: [x[0], 
                                               x[1][0], x[1][1], x[1][2], 
                                               x[1][3], x[1][4], x[1][5], 
                                               x[1][6], x[1][7], x[1][8]])
test_rdd_5 = test_rdd_merge_8.map(lambda x: [x[0], 
                                             x[1][0], x[1][1], x[1][2], 
                                             x[1][3], x[1][4], x[1][5], 
                                             x[1][6], x[1][7], x[1][8]])

In [20]:
print(train_rdd_5.count())
print(train_rdd_5.take(1))

619000
[['d6cf11a892144d6d2b5d953a5dba12033b2041655aaf26647f1f253e079f7951', 45, 32649413.6, '2018-10-06 05:05:00', 'US', 'iOS', 1, 63.0, 12.0, 35.616]]


In [21]:
print(test_rdd_5.count())
print(test_rdd_5.take(1))

619520
[['d6cf11a892144d6d2b5d953a5dba12033b2041655aaf26647f1f253e079f7951', 46, 33233953.52173913, '2018-10-06 05:05:00', 'US', 'iOS', 1, 63.0, 12.0, 35.616]]


In [22]:
test_rdd_4.map(lambda x: x[0]).distinct().count() # Check

619520

## Combine Label to Train

In [39]:
# Impute function
def npImpute(label):
    if label == None:
        return (-1.0, -1.0)
    return label
npImpute(None)

(-1.0, -1.0)

In [44]:
train_label_3 = train_label_2.map(lambda x: (x[0], (x[1], x[2])))
train_rdd_6 = train_rdd_merge_8.leftOuterJoin(train_label_3)\
                               .map(lambda x: (x[0], (x[1][0], npImpute(x[1][1]))))\
                               .map(lambda x: (x[0], sum([x[1][0], x[1][1]], ())))\
                               .map(lambda x: [x[0], 
                                               x[1][0], x[1][1], x[1][2], 
                                               x[1][3], x[1][4], x[1][5], 
                                               x[1][6], x[1][7], x[1][8],
                                               x[1][9], x[1][10]])

In [45]:
print(train_rdd_6.count())
print(train_rdd_6.take(1))

619000
[['eb37a43eb25770c442b54a4943c7f9eb17278328dd0499f2957353dc4cb44a7b', 9, 3560204.222222222, '2018-11-15 09:53:20', 'IN', 'Android OS', 1, 84.0, 0.0, 0.0, -1.0, -1.0]]


### Data Dictionary (Training)
0 - user_bash_id  
1 - number of unique session_id's  
2 - mean of previous sessions (in ms)  
3 - date the user was created  
4 - most frequent country  
5 - most frequent os  
6 - count of unique device id  
7 - max number of events in a session  
8 - total number of purchase  
9 - total amount of dollar in purchase  
10 - label 1: boolean on whether there is a purchase Dec.2 - 8  
11 - label 2: boolean on whether there is a purchase Dec.2 - 15  

In [23]:
# Save train_rdd_6 as train + label

### Data Dictionary (Testing)
0 - user_bash_id  
1 - number of unique session_id's  
2 - mean of previous sessions (in ms)  
3 - date the user was created  
4 - most frequent country  
5 - most frequent os  
6 - count of unique device id  
7 - max number of events in a session  
8 - total number of purchase  
9 - total amount of dollar in purchase  

In [24]:
# Save test_rdd_5 as test